<a href="https://colab.research.google.com/github/Midgraph/AnimationKit-AI/blob/main/Copy_of_AnimationKit_Rife_RealESRGAN_Upscaling_Interpolation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AnimationKit AI - Upscaling & Interpolation using Real-ESRGAN+RIFE

**ALPHA 2: Released 9/4/21**

New features:

- Deflickering (P3)
- Upscaling from individual frames (P2)
- Target length (in seconds) for RIFE interpolation

---


Credits: Motion smoothing conceived from "Zoom animation processing and motion interpolation" added by https://twitter.com/unltd_dream_co. This part of the script uses [RIFE real-time video interpolation](https://github.com/hzwer/arXiv2020-RIFE) to smooth out the resulting video. 

Upscaling uses Real-ESRGAN (https://github.com/xinntao/Real-ESRGAN). A demo notebook for static images can be found here: https://colab.research.google.com/drive/1k2Zod6kSHEvraybHl50Lys0LerhyTMCo?usp=sharing. The demo was based on the following paper: of our paper [''Real-ESRGAN: Training Real-World Blind Super-Resolution with Pure Synthetic Data''](https://arxiv.org/abs/2107.10833).

<img src="https://raw.githubusercontent.com/xinntao/Real-ESRGAN/master/assets/teaser.jpg" width="100%">

Special thanks to @sportsracer48 and everyone on his Discord. If you want closed beta access to the best VQGAN animation notebook on the planet, check out https://www.patreon.com/sportsracer48

---

Testing notes:

Many ffmpeg examples at https://docs.google.com/document/d/12X_2YoCnPPN7B3OsgX39aYyRF8OF-TVStkFTkKhWrx4/edit

Python modules in Colab: https://medium.com/analytics-vidhya/importing-your-own-python-module-or-python-file-into-colab-3e365f0a35ec

Colab Markdown guide: https://colab.research.google.com/notebooks/markdown_guide.ipynb?hl=es-ni#scrollTo=Lhfnlq1Surtk



In [ ]:
# @title Licensed under the MIT License

# Copyright (c) 2021 Katherine Crowson

# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:

# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.

# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN
# THE SOFTWARE.

In [ ]:
!nvidia-smi

# P1: Mount Google Drive & Install Libraries

In [ ]:
#I'm leaving these outside AnimationKit.py for transparency

def installPhase():
  %cd /content/
  installESRGAN()
  %cd /content/
  installRIFE()
  #!pip install ffmpeg-python
  %cd /content/
###

def installESRGAN():
  print("Installing libraries for Real-ESRGAN upscaling.")
  !git clone https://github.com/xinntao/Real-ESRGAN.git
  %cd Real-ESRGAN
  !pip install basicsr
  !pip install facexlib
  !pip install gfpgan
  !pip install -r requirements.txt
  !python setup.py develop
  # Download the pre-trained model(s)
  !wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth -P experiments/pretrained_models
  !wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.2.4/RealESRGAN_x4plus_anime_6B.pth -P experiments/pretrained_models
  !wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.1/RealESRGAN_x2plus.pth -P experiments/pretrained_models
  print("Finished Installing libraries for Real-ESRGAN upscaling.")
###

def installRIFE():
  print("Installing libraries for RIFE motion smoothing.")
  !git clone https://github.com/hzwer/arXiv2020-RIFE RIFE
  !gdown --id 1wsQIhHZ3Eg4_AfCXItFKqqyDMB4NS0Yd
  !7z e RIFE_trained_model_HDv2.zip
  !mkdir /content/RIFE/train_log
  !mv *.pkl /content/RIFE/train_log/
  %cd /content/RIFE/
  !gdown --id 1i3xlKb7ax7Y70khcTcuePi6E7crO_dFc
  !pip3 install -r requirements.txt
  print("Done.")
  print("Finsihed Installing libraries for RIFE motion smoothing.")
###
##
#

'''MARKDOWN
#If RIFE (P2) stops for no reason, go to Runtime>Change Runtime type and set the notebook to "high memory"
'''

#Params
mount_google_drive = True #@param {type:"boolean"}

#Mount Google Drive
if mount_google_drive:
  from google.colab import drive
  drive.mount('/content/drive')

#install dependencies
installPhase()

# P2: Real-ESRGAN Video Upscaling (Somewhat Experimental)

Planned additions: Deflickering, faces option, tiles option

In [ ]:
#@title Rename Files
#@markdown Use this cell to batch rename your image files to the expected format for correct parsing.
individual_frames_path='/content/drive/MyDrive/test/' #@param {type:"string"}
padding_amnt = 5

import os
path = '/path/to/files/'
print(f'Renaming:')
for filename in os.listdir(individual_frames_path):
    prefix, num = filename[:-4].rsplit('_', 1)
    num = num.zfill(padding_amnt)
    new_filename = "out_" + num + ".png"
    print(f'{filename} > {new_filename}')
    os.rename(os.path.join(individual_frames_path, filename), os.path.join(individual_frames_path, new_filename))

In [ ]:
import os, sys, re

def splitFrames():
  #Extract Individual frames
  output_file_path = re.search("^[\/].+\/", mp4_file)
  output_file_path_raw = output_file_path.group(0)
  delsplit = re.search("\/(?:.(?!\/))+$", mp4_file)
  filename = re.sub("^[\/]", "", delsplit.group(0))
  filename_raw = re.sub(".{4}$", "", filename)
  file_extension = re.search(".{3}$", filename)
  file_extension_raw = file_extension.group(0)
  os.environ['inputFile'] = mp4_file
  os.environ['outputPath'] = output_file_path_raw
  #os.environ['startTime'] = start_time
  #os.environ['endTime'] = end_time
  os.environ['frameRate'] = target_fps
  os.environ['fileName'] = filename_raw
  os.environ['fileExtension'] = file_extension_raw
  #ffmpeg
  %cd /content/Real-ESRGAN/
  !ffmpeg -hide_banner -i "$inputFile" -vsync 0 "/content/Real-ESRGAN/upload/out_%05d.png"
  #!ffmpeg -hide_banner -i "$inputFile" -r "$frameRate"/1 "/content/Real-ESRGAN/upload"/frame%04d.png
###

def runUpscale():
  #upload images
  import os
  from google.colab import files
  import shutil
  %cd /content/Real-ESRGAN/
  upload_folder = 'upload'
  result_folder = 'results'
#  if os.path.isdir(upload_folder):
#      shutil.rmtree(upload_folder)
#  if os.path.isdir(result_folder):
#      shutil.rmtree(result_folder)
#  os.mkdir(upload_folder)
#  os.mkdir(result_folder)
  #run upscaler
  #!python inference_realesrgan.py --model_path experiments/pretrained_models/RealESRGAN_x4plus.pth --input upload --netscale $scale_value --outscale $scale_value --half --face_enhance
  !python /content/Real-ESRGAN/inference_realesrgan.py --model_path $model_value --netscale $scale_value --input $frames_dropbox
###
def rebuildFrames():
  #re-encode video from frames
  #results = '/content/Real-ESRGAN/results'
  #!ffmpeg -r 1/5 -pattern_type glob -i '*.png' -c:v libx264 /content/output.mp4   # x264 video
  %cd /content/Real-ESRGAN/results/
  #!ffmpeg -r $target_fps -pattern_type glob -i '*.png' -c:v libx264 $output_path_mp4   # x264 video
  if input_mp4:
      !ffmpeg -r $target_fps -i "out_%05d_out.png" -c:v libx264 $output_path_mp4   # x264 video
  else:
    !ffmpeg -r $target_fps -i "out_%05d_out.png" $output_path_mp4
###
##
#

###PARAMS
#frame_rate = "23.976" #@param {type:"string"}
#start_time = "00:00:00.000"
#end_time = "00:00:05.000" #@param {type:"string"}
#@markdown #Input settings
#@markdown 
input_mp4 = True #@param {type:"boolean"}
mp4_file='/content/drive/MyDrive/test/final3.mp4' #@param {type:"string"}
#@markdown ---
individual_frames_path='y' #@param {type:"string"}
#@markdown If you checkboxed `input_mp4`, `individual_frames_path` will be ignored (and vice-versa).

#@markdown
#@markdown ---
#@markdown #Output settings
output_path_mp4='/content/drive/MyDrive/test/test.mp4' #@param {type:"string"}
target_fps='60'#@param {type:"string"}
model_value='/content/Real-ESRGAN/experiments/pretrained_models/RealESRGAN_x2plus.pth' #@param ['/content/Real-ESRGAN/experiments/pretrained_models/RealESRGAN_x4plus_anime_6B.pth','/content/Real-ESRGAN/experiments/pretrained_models/RealESRGAN_x4plus.pth','/content/Real-ESRGAN/experiments/pretrained_models/RealESRGAN_x2plus.pth'] {type:"string"}
scale_value="2" #@param [2, 4] {type:"string"}

#@markdown `scale_value` should match the model name, eg 4xplus would use scale value of 4
#@markdown Run RIFE below to fill in empty or duplicate frames. (Double, quadruple etc the length of your video)

#
frames_dropbox = "/content/Real-ESRGAN/upload"
%cd /content/
print("Cleaning up from last run...")
!rm -rf "/content/Real-ESRGAN/results"
!rm -rf "/content/Real-ESRGAN/upload"
!mkdir "/content/Real-ESRGAN/results"
!mkdir "/content/Real-ESRGAN/upload"

#handling of individual frames checkbox
if input_mp4:
  splitFrames() #converts video to individual png images
else:
  %cd $individual_frames_path
  #!find -maxdepth 1 -iname '*.png' -exec cp {} /content/Real-ESRGAN/upload \;
  print("Copying frames to Real-ESRGAN/upload...")
  !find -maxdepth 1 -name '*.png' -print0 | xargs -0 cp -t $frames_dropbox
  print("Finished copying frames to Real-ESRGAN/upload.")
#
%cd /content/
runUpscale()
rebuildFrames()


/content
Cleaning up from last run...
/content/Real-ESRGAN
Input #0, mov,mp4,m4a,3gp,3g2,mj2, from '/content/drive/MyDrive/test/final3.mp4':
  Metadata:
    major_brand     : isom
    minor_version   : 512
    compatible_brands: isomiso2avc1mp41
    encoder         : Lavf57.83.100
  Duration: 00:00:01.20, start: 0.000000, bitrate: 1664 kb/s
    Stream #0:0(und): Video: h264 (High 4:4:4 Predictive) (avc1 / 0x31637661), yuv444p, 640x640, 1652 kb/s, 60 fps, 60 tbr, 15360 tbn, 120 tbc (default)
    Metadata:
      handler_name    : VideoHandler
Stream mapping:
  Stream #0:0 -> #0:0 (h264 (native) -> png (native))
Press [q] to stop, [?] for help
Output #0, image2, to '/content/Real-ESRGAN/upload/out_%05d.png':
  Metadata:
    major_brand     : isom
    minor_version   : 512
    compatible_brands: isomiso2avc1mp41
    encoder         : Lavf57.83.100
    Stream #0:0(und): Video: png, rgb24, 640x640, q=2-31, 200 kb/s, 60 fps, 60 tbn, 60 tbc (default)
    Metadata:
      handler_name    : Video

# P2 Motion interpolation (RIFE Smoothing)

"(...) (O)utput can be a bit wobbly, so video interpolation can be used to smooth things out a bit."

New: Type in the number of seconds you want your clip to be!


In [ ]:
def detect_fps():
  fps_ffprobe = !ffprobe -v error -select_streams v -of default=noprint_wrappers=1:nokey=1 -show_entries stream=avg_frame_rate $input_path
  fps_unfinished = [str(i) for i in fps_ffprobe] # Converting integers into strings
  fps_unfinishedTwo = str("".join(fps_unfinished)) # Join the string values into one string
  numbers = re.findall('[0-9]+', fps_unfinishedTwo)
  newNum = numbers[0:1]
  strings = [str(integer) for integer in newNum]
  a_string = "".join(strings)
  fps = int(a_string)
  #print("Detected FPS is",fps)
  return fps
#

def detect_duration():
  duration_ffprobe = !ffprobe -v error -select_streams v:0 -show_entries stream=duration -of default=noprint_wrappers=1:nokey=1 $input_path
  duration_unfinished = [str(i) for i in duration_ffprobe] # Converting integers into strings
  duration_unfinishedTwo = str("".join(duration_unfinished)) # Join the string values into one string
  numbers = re.findall('[0-9]+', duration_unfinishedTwo)
  newNum = numbers[0:1]
  strings = [str(integer) for integer in newNum]
  a_string = "".join(strings)
  duration = int(a_string)
  #print("Detected duration INTEGER (in seconds) is",duration)
  return duration
#

def exp_calc():
  import numpy as np
  a = measured_fps * measured_duration
  b = target_fps * length_in_seconds
  c = b / a
  l = np.log(c) / np.log(2)
  print("Un-rounded --exp is",l)
  x = round(l)
  print("Rounding up to an --exp of ",x)
  return x

%cd /content/RIFE/
# @title Settings
#input_path='/content/output.mp4' #@param {type:"string"}
input_path='/content/drive/MyDrive/VQLIPSE/videos/2blackhole_AnKIT.mp4' #@param {type:"string"}

#autodetect fps


target_fps=120#@param {type:"integer"}
length_in_seconds=60#@param {type:"integer"}
#length_multiplier=3#@param {type:"integer"}
#@markdown
#@markdown
#@markdown ---
#@markdown Note: Your output will look like `original_2x_60fps.mp4` and can be found in your `input_path`.
#@markdown
#@markdown Tip: Save as a high FPS (eg 999) if you plan on doing heavy editing (that way you don't need to re-process at the end.)


measured_fps = detect_fps()
print("Detected average FPS of",input_path,"is",measured_fps)
measured_duration = detect_duration()
print("Detected duration INTEGER (in seconds) is",measured_duration)
exp_value = exp_calc()
print("Warning: Target duration currently rounds to the closest integer.")
!python3 /content/RIFE/inference_video.py --fps=$target_fps --exp=$exp_value --video=$input_path
%cd /content/

# P3: Deflickering & x265 Compression (Experimental, Very Fast)

Todo: Add option for "Auto-pick path" (for people who ran the above step)

Sort of like Handbrake - good for large filesizes. Can turn 500mb files into 200mb files with negligable quality loss. Deflickering is good for deflickering stopmotions, VQGAN_CLIP animations, etc.

Also, too high of compressions might require a decent local GPU to view. Will add h264 toggle in the future~



In [ ]:
def runFF():
  !ffmpeg -i $compress_path $visual_effects -c:v hevc_nvenc -rc vbr -cq $constant_quality -qmin $constant_quality -qmax $constant_quality -b:v 0 $compress_path$outputStr

compress_path='/content/drive/MyDrive/VQLIPSE/videos/2blackhole_AnKIT_4X_120fps.mp4' #@param {type:"string"}
outputStr = '_deflicker-100.mp4' #@param {type:"string"}
enable_visual_effects = True #@param {type:"boolean"}
constant_quality=35#@param {type:"slider", min:20, max:50, step:1}
#animation_tune_toggle = True #@param {type:"boolean"}
#@markdown Default `constant_quality` is `27`, which scrunched a 500mb 40 min video into ~200mb. Higher values = lower filesize, lower quality
#################
#@markdown ---
deflicker_on = True #@param {type:"boolean"}
deflicker_avg_frames=100#@param {type:"slider", min:2, max:129, step:1}
#deblock_toggle = True #@param {type:"boolean"}
################
#@markdown Set `deflicker_avg_frames` filter size in frames. FFmpeg's default is 5. I haven't tested too much but I would try 100.

#@markdown ---


#@markdown Note: Your output will look like `filename_outputStr.mp4` and can be found in your `compress_path`.


#if animation_tune_toggle:
#  tune_value = '-tune animation'

#video effects
if enable_visual_effects:
  visual_effects = '-vf '
  if deflicker_on:
    visual_effects = visual_effects + 'deflicker=s=' + str(deflicker_avg_frames) + ':m=am'
#  if deblock_toggle:
#    x265_params = '-x265-params deblock=-3,-3 '
else:
  visual_effects = ''



runFF()

